In [281]:
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np

In [282]:
im01 = np.array(Image.open('./RobertMixed03.jpg'))/255
im02 = np.array(Image.open('./smallstrelitzia.jpg'))/255
im03 = np.array(Image.open('./smallsunset.jpg'))/255
pixs01 = np.reshape(im01, (im01.shape[0]*im01.shape[1], 3))
pixs02 = np.reshape(im02, (im02.shape[0]*im02.shape[1], 3))
pixs03 = np.reshape(im03, (im03.shape[0]*im03.shape[1], 3))

In [283]:
def init_para(pixels, segments):
    kmeans = KMeans(n_clusters = segments, init = "random", random_state = 0).fit(pixels)
    mu = kmeans.cluster_centers_
    pi = np.ones((segments, 1))/segments
    return mu, pi

In [284]:
def estimation(pixels, mu, pi):
    w = np.zeros((pixels.shape[0], mu.shape[0]))
    for x in range(pixels.shape[0]):
        pixel = np.array([pixels[x]]*10)
        diff = np.subtract(pixel, mu)
        sqrt = np.multiply(diff, diff)
        power = -1/2 * np.sum(sqrt, axis=1)
        weight = np.multiply(np.exp(power), pi.T)
        w[x] = np.divide(weight, np.sum(weight))
    return w

In [285]:
def maximization(pixels, w):
    mu = np.zeros((w.shape[1], pixels.shape[1]))
    pi = np.zeros((w.shape[1], 1))
    for c in range(w.shape[1]):
        w_c = w.T[c]
        mu_num = np.matmul(pixels.T, w_c)
        mu_denom = np.sum(w_c)
        mu[c] = np.divide(mu_num, mu_denom)
        pi[c] = np.divide(mu_denom, pixels.shape[0])
    return mu, pi

In [286]:
def EM(it, pixels, mu, pi):
    if it==0:
        return mu,pi
    else:
        w = estimation(pixels, mu, pi)
        mu, pi = maximization(pixels, w)
        return EM(it-1, pixels, mu, pi)

In [287]:
mu0, pi0 = init_para(pixs01, 10)

In [288]:
mu, pi = EM(5, pixs01, mu0, pi0)